## Importing relevant libraries

In [ ]:
import numpy as np
import scipy.linalg as la
import scipy.spatial as spat
from scipy.stats import unitary_group
from scipy.stats import moment
from scipy.stats import skew, kurtosis
from scipy.optimize import curve_fit
from scipy.linalg import norm
import matplotlib.pyplot as plt
import math
from dataclasses import dataclass

# Libraries for implementing the VQD algorithm
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.primitives import Sampler, Estimator
from qiskit_aer import AerSimulator
from qiskit_algorithms.utils import algorithm_globals
# from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Sampler, Session, Options
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import RealAmplitudes, TwoLocal, EfficientSU2
from qiskit_algorithms.optimizers import *
from qiskit_algorithms.state_fidelities import ComputeUncompute

from qiskit_algorithms.eigensolvers import EigensolverResult, VQD
from qiskit_algorithms import NumPyMinimumEigensolver, VQE

# Import classical optimizers
from qiskit_algorithms.optimizers import SPSA, P_BFGS, COBYLA, IMFIL, SNOBFIT, NELDER_MEAD, SLSQP, NFT, ADAM, POWELL, GradientDescent, BOBYQA

# Import Statevector and SparsePauliOp
from qiskit.quantum_info import SparsePauliOp, Statevector

# Import noise models
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

# Import a fake backend and Qiskit simulators and/or noise libraries
from qiskit_aer import AerSimulator
# from qiskit_aer.primitives import Estimator as AerEstimator 
# from qiskit_aer.noise import NoiseModel

## Helper functions

In [ ]:
def find_probability(eigenvector_raw):
    """
    Purpose: Find the probability associated with each basis of an eigenvector
    Input: eigenvector_raw -> Numpy array documenting the number of times each basis is detected within the eigenvector
    Output: eigenvector_prob -> Numpy array documenting the probability of detecting each basis
    """
    count_total = np.sum(eigenvector_raw)
    eigenvector_prob = eigenvector_raw / count_total
    
    return eigenvector_prob

def find_amplitude(eigenvector_prob):
    """
    Purpose: Finding the probability amplitude of each basis using quantum mechanics
    Input: eigenvector_prob -> Numpy array documenting the probability that each basis is measured
    Output: eigenvector -> Numpy array representing the eigenvector
    """
    eigenvector = np.sqrt(eigenvector_prob)
    return eigenvector

def normalize_eigenvector(vector):
    """
    Purpose: Normalizes a vector such that its norm is 1
    Input: vector -> The vector to be normalized
    Output: vector -> The normalized vector
    """
    L2 = np.sum(np.square(vector))
    vector = vector / np.sqrt(L2)

    return vector

def make_operator_even(op):
    op_new = np.zeros((op.shape[0]//2, op.shape[1]//2))

    for row in range(op_new.shape[0]):
        for col in range(op_new.shape[1]):
            op_new[row, col] = op[row*2, col * 2]

    return op_new

def get_pdf(n, x, dx, L, shift, zeromode_qpe, normalize = True, make_even = False):
    # Function to construct the ground state PDF using the VQSVD zeromode
    
    if not make_even:
        eigenvector = zeromode_qpe
    else:
        eigenvector_old = zeromode_qpe
        eigenvector = np.zeros(n + 1)
        for i in range(len(eigenvector_old)):
            eigenvector[2*i] = eigenvector_old[i]
            
    x0 = x - shift
    
    # Computing the PDF
    y = np.zeros(len(x0))

    for i in range(len(x0)):
        states = state_n(nmax, x0[i], L)
        y[i] += (np.dot(states, eigenvector))
    
    if normalize:
        y = normalize_probability(y, dx)

    return x0, y

def compute_expectation_x_squared_simpson(x, y, n):
    """
    Computes the expectation value of x^2 using Simpson's rule for numerical integration.
    
    Parameters:
    x (array-like): Discrete values of x.
    y (array-like): Corresponding values of the probability density function (PDF) at x.
    
    Returns:
    float: The expectation value of x^2.
    """
    # Ensure x and y are numpy arrays
    x = np.array(x)
    y = np.array(y)
    
    # Compute x^2
    x_squared = x**n
    
    # Check if the number of intervals is even, if not make it even by truncating the last point
    if len(x) % 2 == 0:
        x = x[:-1]
        y = y[:-1]
        x_squared = x_squared[:-1]
    
    # Compute the integral using Simpson's rule
    h = (x[-1] - x[0]) / (len(x) - 1)
    integral = y[0] * x_squared[0] + y[-1] * x_squared[-1] + \
               4 * np.sum(y[1:-1:2] * x_squared[1:-1:2]) + \
               2 * np.sum(y[2:-2:2] * x_squared[2:-2:2])
    integral *= h / 3
    
    return integral

def get_pdf(n, x, dx, L, shift, zeromode_qpe, normalize = True, make_even = False):
    # Function to construct the ground state PDF using the VQSVD zeromode
    
    if not make_even:
        eigenvector = zeromode_qpe
    else:
        eigenvector_old = zeromode_qpe
        eigenvector = np.zeros(n + 1)
        for i in range(len(eigenvector_old)):
            eigenvector[2*i] = eigenvector_old[i]
            
    x0 = x - shift
    
    # Computing the PDF
    y = np.zeros(len(x0))

    for i in range(len(x0)):
        states = state_n(nmax, x0[i], L)
        y[i] += (np.dot(states, eigenvector))
    
    if normalize:
        y = normalize_probability(y, dx)

    return x0, y

# Fidelity measure 1
def get_fidelity(zeromode_classic, zeromode_vqe):
    # Function to compute the infidelity

    overlap = np.dot(np.transpose(zeromode_vqe), zeromode_classic)
    fidelity = 1 - overlap ** 2
    return fidelity

# Fidelity measure 2
def get_similarity(a, b):
    # Function to compute the similarity between 2 zeromodes
    
    numerator = np.abs(np.dot(a.conj().T, b))**2
    denominator = np.linalg.norm(a)**2 * np.linalg.norm(b)**2
    
    return numerator / denominator

def compute_errors(expect_classical, expect_quantum):

    error = np.abs(expect_classical - expect_quantum) / expect_classical
    return error

## VQE implementation

In [ ]:
# VQE run for a given ansatz depth and optimizer
def run_vqe_resource_estimation(matrix, ansatz_depth, optimizer, seed, exact_ground_state):
    
    # Get the Pauli-decomposed form of the operator
    qub_hamiltonian = SparsePauliOp.from_operator(matrix)
    dimension = matrix.shape[0]
    num_qubits = int(np.log2(dimension))
    
    # Define the variational ansatz
    ansatz = RealAmplitudes(num_qubits=num_qubits, reps=ansatz_depth)
    
    # Set up the random initial point
    np.random.seed(seed)
    initial_point = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)
    
    # Initialize the Estimator primitive
    estimator = Estimator()

    # Track optimizer function evaluations using a callback log
    @dataclass
    class VQELog:
        parameters: list
        values: list
        def update(self, count, parameters, mean, _metadata):
            self.values.append(mean)  # Tracks each function evaluation
            self.parameters.append(parameters)
            
    log = VQELog([], [])
    
    # Run VQE with the callback to track function calls
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point, callback=log.update)
    
    # Get the VQE results
    result = vqe.compute_minimum_eigenvalue(qub_hamiltonian)
    length = len(exact_ground_state)
    
    # Get the zeromode
    optimal_params = result.optimal_point
    final_circuit = ansatz.assign_parameters(optimal_params)
    vqe_statevector = Statevector.from_instruction(final_circuit)
    
    # Convert the quantum and classical zeromodes into 4 x 1 arrays
    exact_ground_state = np.array(exact_ground_state).reshape((length, 1))
    vqe_statevector = vqe_statevector.data[:length]  # Slice if exact_ground_state is shorter
    
    zeromode = np.array(vqe_statevector).reshape((length, 1))
    
    # Compute the fidelity measure
    fidelity_value = get_similarity(exact_ground_state, zeromode)
    
    # Total number of function calls is the length of log.values
    num_func_calls = len(log.values)
    
    return zeromode, fidelity_value, num_func_calls

## Estimate quantum resources

In [ ]:
def estimate_resources_FPE(matrix, zeromode_classic, optimizers, num_qubits, target_fidelity_threshold):
    initial_depth = 1
    max_depth = 10

    # Initialize dictionaries to store results for each optimizer
    min_depths = {}
    zeromodes = {}
    gate_counts = {}
    circuit_depths = {}
    function_calls = {}

    for optimizer in optimizers:
        optimizer_name = optimizer.__class__.__name__
        print(f"\nRunning VQE for optimizer: {optimizer_name}")

        current_depth = initial_depth
        converged = False  # Flag to check if convergence occurs

        while current_depth <= max_depth:
            fidelities = []
            func_calls = []
            zeromodes_per_run = []

            # Perform multiple independent VQE runs to calculate average fidelity
            for run in range(10):  # Number of independent runs
                seed = run + 1

                # Run VQE, assuming it returns zeromode, fidelity_value, and num_func_calls
                zeromode, fidelity_value, num_func_calls = run_vqe_resource_estimation(
                    matrix=matrix,
                    ansatz_depth=current_depth,
                    optimizer=optimizer,
                    seed=seed,
                    exact_ground_state=zeromode_classic
                )

                fidelities.append(fidelity_value)
                func_calls.append(num_func_calls)
                zeromodes_per_run.append(zeromode)

            average_fidelity = np.mean(fidelities)
            print(f"Depth {current_depth}: Average fidelity = {average_fidelity}")

            # Check if the average fidelity meets the threshold
            if average_fidelity >= target_fidelity_threshold:
                min_depths[optimizer_name] = current_depth

                # Identify the run with the highest fidelity and its function calls
                max_fidelity_index = np.argmax(fidelities)
                zeromodes[optimizer_name] = zeromodes_per_run[max_fidelity_index]
                function_calls[optimizer_name] = func_calls[max_fidelity_index]
                
                # Calculate gate count and circuit depth for the ansatz at this depth
                ansatz = RealAmplitudes(num_qubits=num_qubits, reps=current_depth)
                decomposed_ansatz = ansatz.decompose()  # Decompose to get actual gate operations
                gate_count_dict = decomposed_ansatz.count_ops()
                total_gates = sum(gate_count_dict.values())
                
                # Store gate counts and circuit depth
                gate_counts[optimizer_name] = total_gates
                circuit_depths[optimizer_name] = decomposed_ansatz.depth()
                
                # Print the gate count, circuit depth, and function calls at optimal depth
                print(f"\nOptimizer: {optimizer_name}")
                print(f"Minimum ansatz depth to meet fidelity threshold: {current_depth}")
                print(f"Total number of gates at optimal depth: {gate_counts[optimizer_name]}")
                print(f"Circuit depth at optimal depth: {circuit_depths[optimizer_name]}")
                print(f"Function calls for the highest fidelity run at optimal depth: {function_calls[optimizer_name]}")
                
                converged = True
                break

            current_depth += 1

        if not converged:
            min_depths[optimizer_name] = "Did not converge"
            zeromodes[optimizer_name] = "Did not converge"
            gate_counts[optimizer_name] = "N/A"
            circuit_depths[optimizer_name] = "N/A"
            function_calls[optimizer_name] = "N/A"
            print(f"{optimizer_name} did not converge within {max_depth} depths.")

    return min_depths, zeromodes, gate_counts, circuit_depths, function_calls